### Preparar ambiente

Primero descomentar las siguientes lineas e instalar PyAthena. Solo se hace la primera vez que se abre la instancia

In [1]:
import sys
!{sys.executable} -m pip install PyAthena

In [2]:
import sys
!{sys.executable} -m pip install pytz

#### Conectar con pyathena

In [3]:
from pyathena.pandas.cursor import PandasCursor
from pyathena import connect
import pandas as pd
import time
# agregar directorio en el cual se almacenan las tablas que se van a consultar
directorio = 's3://iadbprod-csd-hub-analyticaldata/graphdata-mobility-temporal/athena-results/'
# base de datos
bd = 'graphdata' # Database in Glue
tabla_delta = 'march_data_200_files' # Tabla original
cursor = connect(s3_staging_dir = directorio, region_name = 'us-east-1', schema_name = bd, cursor_class = PandasCursor).cursor()

#### Crear tablas por pais

In [4]:
import pytz
from pytz import timezone
from datetime import datetime, timedelta
import numpy as np

Para todos los paises corregir por timestamp, seleccionar primer dia en la ventana, eliminar duplicados y generar estadisticas 

In [5]:
paises = ['BO','EC','CL','VE','AR','CR','DO','SV','GT','HN','NI','PA','PY','PE','UY','BZ','GY','JM','TT','CO','MX','BR']
len(paises)

22

In [6]:
fecha_inicio = '09-28-2020'
fecha_final = '09-30-2020'
fechas = pd.date_range(fecha_inicio, fecha_final)

In [7]:
def get_condicion_partition(fechas):
    condicion = ""
    for fecha in fechas:
        año, mes, dia = str(fecha.date()).split('-')
        condicion += f" ({tabla_delta}.partition_1='{mes}' AND {tabla_delta}.partition_2='{dia}') OR"
    return f"({condicion[:-2]})"

In [8]:
for fecha in fechas:
    day = str(fecha.day)
    if len(day) == 1:
        day = f'0{day}'
    codigo_fecha = str(fecha.month) + day
    print(codigo_fecha)
    fechas_ventana = pd.date_range(fecha, periods=5)
    fecha_ventana_inicio = fechas_ventana[0]
    fecha_ventana_final = fechas_ventana[-1]
    min_dia = fecha_ventana_inicio.day
    min_mes = fecha_ventana_inicio.month
    max_dia = fecha_ventana_final.day
    max_mes = fecha_ventana_final.month
    start_time = time.time()
    tablas_generadas = []
    tiempos_paises = {}
    # huso horario del pais 
    time_pais = time.time()
    pais = 'Todos'
    # nombres de tablas
    tabla_datos_pais = f"{pais}_pings_delta_{codigo_fecha}"
    tabla_usuarios_base = f"{pais}_usuarios_sin_estimacion_de_hogar"
    tabla_lugares_noche = f"{pais}_lugares_noche"
    tabla_lugares_moda_noche = f"{pais}_base_homes_with_duplicates_noche" 
    tabla_lugares_dia = f"{pais}_lugares_dia"
    tabla_lugares_moda_dia = f"{pais}_base_homes_with_duplicates_dia" 
    tabla_usuarios_final = f'{pais}_usuarios_delta_{codigo_fecha}'
    print(f"Creando tabla pings {codigo_fecha}")
    condicion_partition = get_condicion_partition(fechas_ventana)
    query_pings_con_hora = (f'''CREATE TABLE {tabla_datos_pais} 
    WITH (
    external_location = 's3://iadbprod-csd-hub-analyticaldata/graphdata-mobility-temporal/Tablas_pings/{tabla_datos_pais}', 
    format = 'PARQUET', 
    parquet_compression = 'SNAPPY',
    partitioned_by = ARRAY['iso_country_code']) 
    AS SELECT t2.caid, 
            t2.utc_timestamp,
            t2.latitude,
            t2.longitude,
            t2.id_type,
            t2.hour_of_day,
            t2.year,
            t2.month,
            t2.day,
            t2.iso_country_code 
    FROM ( SELECT ROW_NUMBER() OVER ( PARTITION BY caid, utc_timestamp ) row_num,
            t1.*
            FROM (SELECT 
                    t0.caid, 
                    t0.utc_timestamp,
                    t0.latitude,
                    t0.longitude,
                    t0.id_type,
                    EXTRACT(HOUR FROM date_add('hour', utc, from_unixtime(utc_timestamp))) AS hour_of_day,
                    EXTRACT(YEAR FROM date_add('hour', utc, from_unixtime(utc_timestamp))) AS year,
                    EXTRACT(MONTH FROM date_add('hour', utc, from_unixtime(utc_timestamp))) AS month,
                    EXTRACT(DAY FROM date_add('hour', utc, from_unixtime(utc_timestamp))) AS day,
                    t0.iso_country_code 
                 FROM (SELECT td.*, utc_latam.utc 
                       FROM (SELECT * 
                             FROM {tabla_delta} 
                             WHERE {condicion_partition} AND {tabla_delta}.iso_country_code in 
                             ('BO','EC','CL','VE','AR','CR','DO','SV','GT','HN',
                             'NI','PA','PY','PE','UY','BZ','GY','JM','TT','CO','MX','BR')) AS td 
                       INNER JOIN utc_latam 
                     ON td.iso_country_code=utc_latam.country_code) AS t0) as t1
            WHERE t1.day={min_dia}) AS t2 
    WHERE t2.row_num=1''')
    cursor.execute(query_pings_con_hora)
    time.sleep(60)
    while (not cursor.has_result_set):
        time.sleep(1)
    cursor.close()
    tablas_generadas.append(tabla_datos_pais)

    # Crear tabla de usuarios base o activos
    UTC_NOCHE = 22 # hora en la que comienza la noche 
    UTC_MAÑANA = 6 # hora en la que comienza el dia 
    query_usuarios_base = (f"CREATE VIEW {tabla_usuarios_base} "
    " AS SELECT caid, "
            "iso_country_code, "
            "MAX(id_type) as id_type, "              
            "COUNT(*) as n_obs,  "
            f" SUM(CASE WHEN hour_of_day >= {UTC_NOCHE} then 1 ELSE 0 END) as n_obs_noche, "
            f" SUM(CASE WHEN hour_of_day <= {UTC_MAÑANA} then 1 ELSE 0 END) as n_obs_madrugada, "
            "year, month, day " 
    f"FROM {tabla_datos_pais} "
    " GROUP BY iso_country_code, caid, year, month, day ")
    cursor.execute(query_usuarios_base)
    time.sleep(6)
    while (not cursor.has_result_set):
        time.sleep(1)
    cursor.close()
    # Crear view con los lugares visitados entre las 00:00 y las 06:00 hora local
    query_time_spent = (
    f"CREATE VIEW {tabla_lugares_noche} AS "
    f"SELECT iso_country_code, "
            "caid, "
            "year, "
            "month, "
            "day, "
            "ROUND(latitude,3) as round_lat,"
            "ROUND(longitude,3) as round_long, "
            f"COUNT(hour_of_day) as n_pings_por_ubicacion, "
            "ROUND(longitude,3)*ROUND(latitude,3) as sum_posicion "
    f"FROM {tabla_datos_pais} "
    f"WHERE hour_of_day>={UTC_NOCHE} OR hour_of_day<={UTC_MAÑANA} " # OJO que lo cambie a OR Si solo se quiere de las 12 de la noche entonces no es necesario kas dos condiciones
    "GROUP BY iso_country_code, caid, year, month, day, ROUND(latitude,3), ROUND(longitude,3)") # modificar query para que funcione con tablas de athena
    cursor.execute(query_time_spent) # hace la consulta y hace la conexión a la tabla resultante
    time.sleep(6)
    while (not cursor.has_result_set):
        time.sleep(1)
    cursor.close()    
    # Crear view con los lugares visitados en las 24h
    query_time_spent = (
    f"CREATE VIEW {tabla_lugares_dia} AS "
    f"SELECT iso_country_code, "
            "caid, "
            "year, "
            "month, "
            "day, "
            "ROUND(latitude,3) as round_lat, "
            "ROUND(longitude,3) as round_long, "
            "COUNT(hour_of_day) as n_pings_por_ubicacion, "
            "ROUND(longitude,3)*ROUND(latitude,3) as sum_posicion "
    f"FROM {tabla_datos_pais} "
    "GROUP BY iso_country_code, caid, year, month, day, ROUND(latitude,3), ROUND(longitude,3)") # modificar query para que funcione con tablas de athena
    cursor.execute(query_time_spent) # hace la consulta y hace la conexión a la tabla resultante
    time.sleep(6)
    while (not cursor.has_result_set):
        time.sleep(1)
    cursor.close()    
    query_moda_de_la_noche = (
        f"CREATE VIEW {tabla_lugares_moda_noche} AS "
        "SELECT  t.iso_country_code, "
                "t.caid, "
                "t.year, "
                "t.month, "
                "t.day, "
                "MAX(t.n_pings_por_ubicacion) as n_obs, "
                "MAX(t.sum_posicion) as max_sum_posicion "
        f"FROM ("
            f"SELECT {tabla_lugares_noche}.* "
            "FROM ("
                "SELECT iso_country_code, "
                        "caid, "
                        "year, "
                        "month, "
                        "day, "
                        "MAX(n_pings_por_ubicacion) as frecuencia_del_hogar  "
                f"FROM {tabla_lugares_noche} "
                "GROUP BY iso_country_code, caid, year, month, day) as freq_hogar "
                f"LEFT JOIN {tabla_lugares_noche} "
                f"ON {tabla_lugares_noche}.iso_country_code=freq_hogar.iso_country_code "
                f"AND {tabla_lugares_noche}.caid=freq_hogar.caid "
                f"AND {tabla_lugares_noche}.year=freq_hogar.year "
                f"AND {tabla_lugares_noche}.month=freq_hogar.month "
                f"AND {tabla_lugares_noche}.day=freq_hogar.day "
                f"AND {tabla_lugares_noche}.n_pings_por_ubicacion=freq_hogar.frecuencia_del_hogar) AS t "
        f"GROUP BY t.iso_country_code, t.caid, t.year, t.month, t.day")
    cursor.execute(query_moda_de_la_noche)
    time.sleep(6)
    while (not cursor.has_result_set):
        time.sleep(1)
    cursor.close()
    
    query_moda_del_dia = (
        f"CREATE VIEW {tabla_lugares_moda_dia} AS "
        f"SELECT t.iso_country_code, "
                "t.caid, "
                "t.year, "
                "t.month, "
                "t.day, "
                "MAX(t.n_pings_por_ubicacion) as n_obs, "
                "MAX(t.sum_posicion) as max_sum_posicion "
        f"FROM ("
            f"SELECT {tabla_lugares_dia}.* "
            "FROM ("
                "SELECT iso_country_code, "
                        "caid, "
                        "year, "
                        "month, "
                        "day, "
                        "MAX(n_pings_por_ubicacion) as frecuencia_del_hogar  "
                f"FROM {tabla_lugares_dia} "
                "GROUP BY iso_country_code, caid, year, month, day ) as freq_hogar "
                f"LEFT JOIN {tabla_lugares_dia} "
                f"ON {tabla_lugares_dia}.iso_country_code=freq_hogar.iso_country_code "
                f"AND {tabla_lugares_dia}.caid=freq_hogar.caid "
                f"AND {tabla_lugares_dia}.year=freq_hogar.year "
                f"AND {tabla_lugares_dia}.month=freq_hogar.month "
                f"AND {tabla_lugares_dia}.day=freq_hogar.day "
                f"AND {tabla_lugares_dia}.n_pings_por_ubicacion=freq_hogar.frecuencia_del_hogar) AS t "
        f"GROUP BY t.iso_country_code, t.caid, t.year, t.month, t.day")
    cursor.execute(query_moda_del_dia)
    time.sleep(6)
    while (not cursor.has_result_set):
        time.sleep(1)
    cursor.close()
    print(f"Creando tabla usuarios {codigo_fecha}")
    query_hogares = (f"CREATE TABLE {tabla_usuarios_final}  "
                     "WITH ("
    f" external_location = 's3://iadbprod-csd-hub-analyticaldata/graphdata-mobility-temporal/Tablas_usuarios/{tabla_usuarios_final}', " 
    " format = 'PARQUET', " 
    " parquet_compression = 'SNAPPY', "
    " partitioned_by = ARRAY['iso_country_code']) AS " 
    f"SELECT t.caid, "
            "t.id_type, "
            "t.n_obs, "
            "t.n_obs_noche, "
            "t.n_obs_madrugada, "
            "t.lat_hogar, "
            "t.long_hogar, "
            "t.n_obs_hogar, "
            "t_dia.round_lat as lat_mas_visitado, "
            "t_dia.round_long as long_mas_visitado, "
            "t_dia.n_pings_por_ubicacion as n_obs_mas_visitado, "
            "t.year, "
            "t.month, "
            "t.day, "
            "t.iso_country_code "
    f"FROM ("
            "SELECT t_u.caid,"
                     "t_u.iso_country_code,"
                     "t_u.id_type, "
                     "t_u.n_obs,"
                     "t_u.n_obs_noche, "
                     "t_u.n_obs_madrugada, "
                     "t_noche.round_lat as lat_hogar, "
                     "t_noche.round_long as long_hogar, "
                     "t_noche.n_pings_por_ubicacion as n_obs_hogar, "
                     "t_u.year, "
                     "t_u.month, "
                     "t_u.day  "
            f"FROM {tabla_usuarios_base} as t_u LEFT JOIN ("
                    f"SELECT {tabla_lugares_noche}.* "
                    f"FROM {tabla_lugares_moda_noche} "
                        f"LEFT JOIN {tabla_lugares_noche} "
                        f"ON {tabla_lugares_moda_noche}.iso_country_code={tabla_lugares_noche}.iso_country_code "
                        f"AND {tabla_lugares_moda_noche}.caid={tabla_lugares_noche}.caid "
                        f"AND {tabla_lugares_moda_noche}.n_obs={tabla_lugares_noche}.n_pings_por_ubicacion "
                        f"AND {tabla_lugares_moda_noche}.year ={tabla_lugares_noche}.year "
                        f"AND {tabla_lugares_moda_noche}.month ={tabla_lugares_noche}.month "
                        f"AND {tabla_lugares_moda_noche}.day ={tabla_lugares_noche}.day "
                    f"WHERE {tabla_lugares_moda_noche}.max_sum_posicion ={tabla_lugares_noche}.sum_posicion "
                    ") as t_noche "
                f"ON  t_u.iso_country_code=t_noche.iso_country_code "
                "AND  t_u.caid=t_noche.caid " 
                "AND  t_u.year=t_noche.year " 
                "AND t_u.month=t_noche.month "
                "AND t_u.day=t_noche.day) AS t "
        "LEFT JOIN ("
            f"SELECT {tabla_lugares_dia}.* "
            f"FROM {tabla_lugares_moda_dia} "
                f"LEFT JOIN {tabla_lugares_dia} "
                f"ON {tabla_lugares_moda_dia}.iso_country_code={tabla_lugares_dia}.iso_country_code "
                f"AND {tabla_lugares_moda_dia}.caid={tabla_lugares_dia}.caid "     
                f"AND {tabla_lugares_moda_dia}.n_obs={tabla_lugares_dia}.n_pings_por_ubicacion "
                f"AND {tabla_lugares_moda_dia}.year ={tabla_lugares_dia}.year "
                f"AND {tabla_lugares_moda_dia}.month ={tabla_lugares_dia}.month "
                f"AND {tabla_lugares_moda_dia}.day ={tabla_lugares_dia}.day "
            f"WHERE {tabla_lugares_moda_dia}.max_sum_posicion={tabla_lugares_dia}.sum_posicion "
        ") as t_dia "
        f"ON t.iso_country_code=t_dia.iso_country_code AND t.caid=t_dia.caid AND  t.year=t_dia.year AND t.month=t_dia.month AND t.day=t_dia.day" )

    cursor.execute(query_hogares)
    time.sleep(15)
    while (not cursor.has_result_set):
        time.sleep(1)
    cursor.close()
    tablas_generadas.append(tabla_usuarios_final)

    for tabla in [ tabla_usuarios_base, tabla_lugares_noche, tabla_lugares_dia, tabla_lugares_moda_noche, tabla_lugares_moda_dia]:
        query = (f"DROP VIEW {tabla}")
        cursor.execute(query)

    tiempo = (time.time()-time_pais)/60
    tiempos_paises[pais] = tiempo
    print((time.time()-start_time)/60)
    print(f"Pais {pais} terminado")
    time.sleep(60)

928
Creando tabla pings 928
Creando tabla usuarios 928
9.682981967926025
Pais Todos terminado
929
Creando tabla pings 929
Creando tabla usuarios 929
10.209870008627574
Pais Todos terminado
930
Creando tabla pings 930
Creando tabla usuarios 930
9.853474199771881
Pais Todos terminado


#### Eliminar tablas generadas

pais = 'Todos'
try:
    fecha_inicio = '03-01-2020'
    fecha_final = '06-25-2020'
    fechas = pd.date_range(fecha_inicio, fecha_final)
    for fecha in fechas:
        day = str(fecha.day)
        if len(day) == 1:
            day = f'0{day}'
        codigo_fecha = str(fecha.month) + day
        print(codigo_fecha)
        
        tabla_datos_pais = f"{pais}_pings_delta_{codigo_fecha}"
        query = f"DROP TABLE {tabla_datos_pais}"
        cursor.execute(query)
        
        tabla_datos_pais = f"{pais}_usuarios_delta_{codigo_fecha}"
        query = f"DROP TABLE {tabla_datos_pais}"
        cursor.execute(query)

        tabla_datos_pais = f"{pais}_usuarios_delta_dist_{codigo_fecha}"
        query = f"DROP TABLE {tabla_datos_pais}"
        cursor.execute(query)
        
        tabla_datos_pais = f"{pais}_usuarios_delta_dist_corregida_{codigo_fecha}"
        query = f"DROP TABLE {tabla_datos_pais}"
        cursor.execute(query)

except:
    print(f"error {pais}")

try:
    fecha_inicio = '06-14-2020'
    fecha_final = '06-25-2020'
    fechas = pd.date_range(fecha_inicio, fecha_final)
    for fecha in fechas:
        day = str(fecha.day)
        if len(day) == 1:
            day = f'0{day}'
        codigo_fecha = str(fecha.month) + day
        print(codigo_fecha)
        
        tabla_datos_pais = f"base_nuevos_{codigo_fecha}"
        query = f"DROP TABLE {tabla_datos_pais}"
        cursor.execute(query)
        
except:
    print(f"error {pais}")